In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(context="poster")
import ipywidgets
import yt
import glob
import os
import warnings

import numpy as np

from astropy import constants as const
from astropy import units as u

M_solar = const.M_sun.cgs.value
gamma = 5/3

@yt.derived_field(name="pressure", units="g  / s**2 / cm")
def _pressure(field, data):
    return (gamma-1) * data["thermal_energy"] * data["density"]

# Warning
This file is very much a work-in-progress.

To do:
 - implement a shock-finder
 - only get energy within the remnant

# Overview

In [ ]:
SN_times = np.array([
    3e+10,
    1.99471e+13,
    8.02558e+13,
    9.92043e+13,
    1.22073e+14,
    2.36641e+14,
    2.46039e+14,
    4.41575e+14,
    5.08510e+14,
    8.57144e+14,
    9.02104e+14,
    ]) / (u.Myr.to(u.s))

In [ ]:
time_unit = (u.pc / (u.km/u.s) ).to(u.Myr)

unit_base = {
    "length":(1.0, "pc"),
    "time":(time_unit, "Myr"),
    "mass":(1.0, "Msun")
}

In [ ]:
# filename = "../output/cluster/snapshot_000.hdf5"
# filename = "../ICs/cluster/cluster_ics.hdf5"
snapshot_dir = "../output/cluster/"

n_files_ready = len(glob.glob(os.path.join(snapshot_dir, "snapshot_*.hdf5")))
if n_files_ready == 0:
    raise FileNotFoundError("No snapshots found in {}".format(snapshot_dir))

ts = yt.load(os.path.join(snapshot_dir,"snapshot_0??.hdf5"),
             unit_base=unit_base)

print("Loaded {} snapshots".format(len(ts)))

times = np.array([ts[i].current_time.convert_to_cgs() 
                  for i in range(len(ts))]) / u.Myr.to(u.s)


In [ ]:
print( "Length unit: ", ts[0].length_unit)
print( "Time unit: ", ts[0].time_unit)
print( "Mass unit: ", ts[0].mass_unit)
print( "Velocity unit: ", ts[0].velocity_unit)

# Plot Global Quantities

In [ ]:
ds = ts[0]

In [ ]:
thermal_energies = np.empty(len(ts))
kinetic_energies = np.empty(len(ts))

for i,ds in enumerate(ts):
    dd = ds.all_data()
    
    thermal_energies[i] = (dd["all", "InternalEnergy"] \
                         * dd["all", "Masses"]).sum()
    
    kinetic_energies[i] = 0.5*(  dd["all", "particle_velocity_magnitude"]**2 \
                               * dd["all", "Masses"]).sum()
    
total_energies = thermal_energies + kinetic_energies

In [ ]:
sns.rugplot(times, color="k", linewidth=3)
plt.plot(times, kinetic_energies)
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$E_\mathrm{kin}$ $[\mathrm{ergs}]$")

In [ ]:
sns.rugplot(times, color="k", linewidth=3)
plt.plot(times, thermal_energies)
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$E_\mathrm{int}$ $[\mathrm{ergs}]$")

In [ ]:
sns.rugplot(times, color="k", linewidth=3)
plt.plot(times, total_energies)
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$E_\mathrm{total}$ $[\mathrm{ergs}]$")

To do: remove the contribution from cooling outside the remnant

In [ ]:
sns.rugplot(times, color="k", linewidth=3)
plt.plot(times, total_energies - total_energies[0])
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$\Delta E_\mathrm{total}$ $[\mathrm{ergs}]$")

## Mass Plots

In [ ]:
total_mass = np.empty(len(ts))

for i,ds in enumerate(ts):
    dd = ds.all_data()
    
    total_mass[i] = (dd["all", "Masses"]).sum()
    
total_mass /= M_solar

In [ ]:
sns.rugplot(times, color="k", linewidth=3)
plt.plot(times, total_mass - total_mass[0])
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$\Delta M$ $[M_\odot]$")

## Momentum Plots

In [ ]:
radial_momentum = np.empty(len(ts))

for i,ds in enumerate(ts):
    dd = ds.all_data()
    
    radial_momentum[i] = (  dd["all", "particle_radial_velocity"] \
                          * dd["all", "Masses"]).sum()


In [ ]:
sns.rugplot(times, color="k", linewidth=3)
plt.plot(times, radial_momentum / (100 * M_solar * 1e5))
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$p$ $[100 M_\odot$ $\mathrm{km}$ $\mathrm{s}^{-1}]$")
plt.ylim(ymin=0)

# Plot Snapshot Views

In [ ]:
def show_projected_density(i):
    ds = ts[i]
    
    p = yt.ProjectionPlot(ds, "x", ("gas","density"))
    p.set_cmap(field="density", cmap="viridis")
    p.annotate_timestamp(corner="upper_left", draw_inset_box=True)
    
    t = ds.current_time.convert_to_cgs().value / u.Myr.to(u.s)
    N_SNe_so_far = np.sum(t > SN_times)
    p.annotate_text((.8,.94), 
                    "N_SNe: {}".format(N_SNe_so_far),
                    coord_system="axis",
                    inset_box_args={"facecolor":"darkslategray",
                                       "alpha":0.9},
                   )
    p.show()
    
ipywidgets.interact(show_projected_density,
                i=ipywidgets.IntSlider(min=0,
                                       max=len(ts)-1,
                                       value=0))

In [ ]:
def show_sliced_field(i, field):
    ds = ts[i]
    
    s = yt.SlicePlot(ds, "z", ("gas", field))
    s.set_cmap(field=field, cmap="viridis")
    s.annotate_timestamp(corner="upper_left", draw_inset_box=True)
    t = ds.current_time.convert_to_cgs().value / u.Myr.to(u.s)
    N_SNe_so_far = np.sum(t > SN_times)
    s.annotate_text((.8,.94), 
                    "N_SNe: {}".format(N_SNe_so_far),
                    coord_system="axis",
                    inset_box_args={"facecolor":"darkslategray",
                                       "alpha":0.9},
                   )
    s.show()
    
ipywidgets.interact(show_sliced_field,
                i=ipywidgets.IntSlider(min=0,
                                       max=len(ts)-1,
                                       value=0),
                field = ipywidgets.Dropdown(options=[
                                                "density",
                                                "temperature",
                                                "pressure",
                                                "velocity_magnitude"
                                            ],
                                            value="density"))

# Profiles

In [ ]:
from astropy.convolution import convolve, Gaussian1DKernel
gauss_kernel = Gaussian1DKernel(2)

def show_profile(i, field):
    ds = ts[i]
    sp = ds.sphere("c", (225, "pc"))
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        pp = yt.create_profile(sp, 
                               "radius", [field, "ones"], 
                               weight_field="cell_volume",
                               units = {"radius":"pc"},
                               logs = {"radius":False},
                               n_bins=256
        )
    mask = pp["ones"] > .5 # filter out bins with no particles within
#     plt.plot(pp.x.value[mask], convolve(pp[field][mask], gauss_kernel, boundary="extend"), label="smoothed")
    plt.plot(pp.x.value[mask], pp[field][mask])

    plt.yscale("log")
#     if field in {"density"}:
#         plt.ylim(1e-28, 1e-23)


    
ipywidgets.interact(show_profile,
                i=ipywidgets.IntSlider(min=0,
                                       max=len(ts)-1,
                                       value=0),
                field = ipywidgets.Dropdown(options=[
                                                "density",
                                                "temperature",
                                                "pressure",
                                                "velocity_magnitude",
                                                "radial_velocity",
                                            ],
                                            value="density"))

# Double check hdf5

In [ ]:
import h5py

In [ ]:
filename = "../output/cluster/snapshot_000.hdf5"
# filename = "../ICs/cluster/cluster_ics.hdf5"

f = h5py.File(filename)

In [ ]:
f["Header"].attrs["BoxSize"]

In [ ]:
for key in f["Header"].attrs:
    print(key, ":", f["Header"].attrs[key])

In [ ]:
for key in f["PartType0"].attrs:
    print(key, ":", f["PartType0"].attrs[key])

In [ ]:
for key in f.attrs:
    print(key, ":", f.attrs[key])

In [ ]:
for key in f["PartType0"].keys():
    print(key, ":", f["PartType0"][key])